# Code for training the holistic model

## Import Libraries 


In [337]:
import tensorflow as tf
import numpy as np
import os
from datetime import datetime

from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard
from keras.optimizers import Adam


from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

## Setup Variables

In [338]:
FOLDER_NAME = 'dataset'
ALL_CLASSES = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [339]:
# Create label map, representing each class as a number
label_map = {}
for (root, folders, files) in os.walk(FOLDER_NAME):
    for foldername in folders:
        if foldername in ALL_CLASSES:
            label_map[foldername] = ALL_CLASSES.index(foldername)

print(label_map)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [340]:
# Get all datset data with its label and put it in a list
sequence, label = [], []
target_length = 14
for (root, folders, files) in os.walk(FOLDER_NAME):
    total_file = 0
    for filename in files:
        file_path = os.path.join(os.path.relpath(
            root, FOLDER_NAME), filename)
        if (filename.endswith('.npy') and os.path.split(file_path)[0] in ALL_CLASSES):
            res = np.load(f'{FOLDER_NAME}/{file_path}')
            for _ in range(target_length-res.shape[0]):
                res = np.vstack((res, res[-1, :]))
            # res = res[:, -126:]
            sequence.append(np.array(res))
            label.append(label_map[os.path.basename(root[-1])])
            total_file += 1
    print(f"Total files: {total_file} --- {root}")

print(np.array(sequence).shape)
print(np.array(label).shape)

Total files: 0 --- dataset
Total files: 50 --- dataset\a
Total files: 50 --- dataset\b
Total files: 50 --- dataset\c
Total files: 50 --- dataset\d
Total files: 50 --- dataset\e
Total files: 50 --- dataset\f
Total files: 50 --- dataset\g
Total files: 50 --- dataset\h
Total files: 50 --- dataset\i
Total files: 50 --- dataset\j
Total files: 50 --- dataset\k
Total files: 50 --- dataset\l
Total files: 50 --- dataset\m
Total files: 50 --- dataset\n
Total files: 50 --- dataset\o
Total files: 50 --- dataset\p
Total files: 50 --- dataset\q
Total files: 50 --- dataset\r
Total files: 50 --- dataset\s
Total files: 50 --- dataset\t
Total files: 50 --- dataset\u
Total files: 50 --- dataset\v
Total files: 50 --- dataset\w
Total files: 50 --- dataset\x
Total files: 50 --- dataset\y
Total files: 50 --- dataset\z
(1300, 14, 1662)
(1300,)


In [341]:
# Dataset duplication if necessary, 1 means no duplication
n = 2
sequence = np.concatenate([sequence] * n, axis=0)
label = np.concatenate([label] * n, axis=0)


print(np.array(sequence).shape)
print(np.array(label).shape)

(2600, 14, 1662)
(2600,)


In [342]:
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Training Data

In [343]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sequence), tf.keras.utils.to_categorical(
    np.array(label).astype(int), num_classes=np.array(ALL_CLASSES).shape[0], dtype='float32'), test_size=0.2)

print(X_train.shape, X_test.shape)


(2080, 14, 1662) (520, 14, 1662)


In [344]:
training_phase = "holistic-1662-tanh-lr-0001-dupli-2x-" + datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join('Logs', training_phase)
tb_callback = TensorBoard(log_dir=log_dir)

print(log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True,
          # activation='tanh', input_shape=(14, 126)))
          activation='tanh', input_shape=(14, 1662)))
model.add(LSTM(64, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(np.array(ALL_CLASSES).shape[0], activation='softmax'))

learning_rate = 0.0001
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

Logs\holistic-1662-tanh-lr-0001-dupli-2x-20240802-090903


In [345]:
model.fit(X_train, y_train, epochs=80, callbacks=[
          tb_callback])

Epoch 1/80
65/65 [==============================] - 5s 17ms/step - loss: 3.2194 - categorical_accuracy: 0.0673
Epoch 2/80
65/65 [==============================] - 1s 13ms/step - loss: 3.1010 - categorical_accuracy: 0.0784
Epoch 3/80
65/65 [==============================] - 1s 13ms/step - loss: 3.0029 - categorical_accuracy: 0.0784
Epoch 4/80
65/65 [==============================] - 1s 22ms/step - loss: 2.9281 - categorical_accuracy: 0.0889
Epoch 5/80
65/65 [==============================] - 1s 13ms/step - loss: 2.8348 - categorical_accuracy: 0.1005
Epoch 6/80
65/65 [==============================] - 2s 24ms/step - loss: 2.7429 - categorical_accuracy: 0.0981
Epoch 7/80
65/65 [==============================] - 1s 23ms/step - loss: 2.6383 - categorical_accuracy: 0.1269
Epoch 8/80
65/65 [==============================] - 1s 12ms/step - loss: 2.4753 - categorical_accuracy: 0.2062
Epoch 9/80
65/65 [==============================] - 1s 12ms/step - loss: 2.3611 - categorical_accuracy: 0.2308
E

In [346]:
model.summary()


Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_66 (LSTM)              (None, 14, 64)            442112    
                                                                 
 lstm_67 (LSTM)              (None, 14, 64)            33024     
                                                                 
 lstm_68 (LSTM)              (None, 64)                33024     
                                                                 
 dense_66 (Dense)            (None, 64)                4160      
                                                                 
 dense_67 (Dense)            (None, 32)                2080      
                                                                 
 dense_68 (Dense)            (None, 26)                858       
                                                                 
Total params: 515,258
Trainable params: 515,258
Non-t

In [347]:
model.save(f'{log_dir}/action.h5')

In [348]:
res = model.predict(X_test)

17/17 [==============================] - 1s 8ms/step


In [349]:

import sys
import io

# Redirect stdout to a string buffer
old_stdout = sys.stdout
sys.stdout = buffer = io.StringIO()

# Assume X_test and y_test are your test dataset
# Make predictions
y_pred = model.predict(X_test)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)  # if y_test is one-hot encoded

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred_classes)

# Calculate precision, recall, and F1 score for each class
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Detailed classification report
report = classification_report(y_true, y_pred_classes)

# Restore stdout
sys.stdout = old_stdout

# Get the contents of the buffer
output = buffer.getvalue()

# Save the output to a text file
with open('classification_results.txt', 'w') as f:
    f.write(output)

# Optionally, print the output to console as well
print(output)

17/17 [==============================] - 0s 7ms/step
Accuracy: 0.8788461538461538
Precision: 0.8852867803655915
Recall: 0.8788461538461538
F1 Score: 0.8780779553907957



In [350]:
actions = np.array(ALL_CLASSES)
testing = 13
print(actions[np.argmax(res[testing])], actions[np.argmax(y_test[testing])])

r r
